In [1]:
from utils.preprocess import JSONLoader
fields = ['business_id','categories','city']
# fields = None
city = ['Toronto']
categories_all =['Restaurants', 'Food', 'Sandwiches', 'Fast Food', 'American (Traditional)', 'Pizza',
                 'Italian', 'Burgers', 'Mexican','Breakfast & Brunch', 'American (New)', 'Chinese', 'Specialty Food', 'Bakeries', 'Desserts',
                 'Japanese', 'Ice Cream & Frozen Yogurt', 'Chicken Wings', 'Seafood', 'Salad', 'Sushi Bars',
                 'Asian Fusion', 'Thai', 'Indian']

business = 'business.json'
data_dir = 'data/dataset'
jl = JSONLoader(business, data_dir, fields = fields)
jl.set_condition(city=city, categories=categories_all)
f_b, all_restaurants = jl.sample(1000000)
print(len(all_restaurants))
all_rest_id = set([restaurant[0] for restaurant in all_restaurants])
print(len(all_rest_id))

8627
8627


In [2]:
# Load the reviews
review = 'review.json'
data_dir = 'data/dataset'
fields = ['user_id','business_id','stars','text' ]
jl = JSONLoader(review, data_dir, fields = fields)
jl.set_condition(business_id = all_rest_id)

f_, all_reviews = jl.sample(1000000)
len(all_reviews)

69192

In [3]:
categories =['Seafood', 'Burgers']

rest_id = {}

for category in categories:
    jl = JSONLoader(business, data_dir, fields = fields)
    jl.set_condition(city=city, categories=[category])
    f_b, categorized_restaurants = jl.sample(1000000)
#     print(len(categorized_restaurants))
    rest_id[category] = set([restaurant[0] for restaurant in categorized_restaurants])
#     print(len(rest_id[category]))

In [4]:
# Load the reviews
categorized_reviews = {}
review = 'review.json'
data_dir = 'data/dataset'
fields = ['user_id','business_id','stars','text']

for category in categories:
    jl = JSONLoader(review, data_dir, fields = fields)
    jl.set_condition(business_id = rest_id[category])
    f_, categorized_reviews[category] = jl.sample(1000000)
    print(len(categorized_reviews[category]))

3278
3005


In [174]:
# To raise the accuracy, we have to consider plural.

def toPlural(word):
    if word.endswith('y'):
        return word[:-1] + 'ies'
    elif word[-1] in 'sx' or word[-2:] in ['sh', 'ch']:
        return word + 'es'
    elif word.endswith('an'):
        return word[:-2] + 'en'
    else:
        return word + 's'

In [164]:
import re, string

def eliminatePunctuation(text):
    out = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    return out

In [166]:
# For particular review texts

categorized_review_texts = {}
categorized_texts_processed = {}
for category in categories:
    categorized_review_texts[category] = [review[3] for review in categorized_reviews[category]]
    categorized_texts_processed[category] = [[word for word in eliminatePunctuation(text_review).lower().split() if word not in stopwords] for text_review in categorized_review_texts[category]]

In [194]:
concatenated_categorized_texts_processed = {}
for category in categories:
    concatenated_categorized_texts_processed[category] = [sum(categorized_texts_processed[category], [])]

In [165]:
# For all review texts
from nltk.corpus import stopwords

all_text_reviews = [review[3] for review in all_reviews]

stopwords.words('english')
stopwords = stopwords.words('english')

texts = [[word for word in eliminatePunctuation(text_review).lower().split() if word not in stopwords] for text_review in all_text_reviews]

In [167]:
from gensim import corpora

all_dictionary = corpora.Dictionary(texts)

In [168]:
'''
testing, please just skip this part.
'''
print(all_dictionary[7])  # just for test
print(all_dictionary.token2id['would']) # just for test

would
7


In [169]:
import gensim.models as gm
tfidf = gm.TfidfModel(corpus)

In [170]:
# TF-IDF per review text per category

categorized_corpus = {}
categorized_tfidf = {}

for category in categories:
    categorized_corpus[category] = [all_dictionary.doc2bow(text) for text in categorized_texts_processed[category]]
    categorized_tfidf[category] = tfidf[categorized_corpus[category]]

In [172]:
print("# of review text under Seafood = ", len(categorized_tfidf['Seafood']))
print("# of review text under Burgers = ", len(categorized_tfidf['Burgers']))

# of review text under Seafood =  3278
# of review text under Burgers =  3005


In [196]:
# TF-IDF per category by being concatenated

concatenated_categorized_corpus = {}
concatenated_categorized_tfidf = {}

for category in categories:
    concatenated_categorized_corpus[category] = [all_dictionary.doc2bow(text) for text in concatenated_categorized_texts_processed[category]]
    concatenated_categorized_tfidf[category] = tfidf[concatenated_categorized_corpus[category]]

In [222]:
# By category, display the words according to tf-idf

import operator

tfidf_under_category = {}
words_sorted_by_tfidf = {}
just = {}
for category in categories:
    intermediate = {}
    for entry in concatenated_categorized_tfidf[category][0]:
        intermediate[all_dictionary[entry[0]]] = entry[1]
    words_sorted_by_tfidf[category] = [ word for (word, value) in sorted(intermediate.items(), key=operator.itemgetter(1), reverse=True)]
    print(category, words_sorted_by_tfidf[category][:20])

Seafood ['food', 'place', 'one', 'great', 'lobster', 'really', 'seafood', 'fried', 'good', 'like', 'back', 'much', 'little', 'came', 'meal', 'crab', 'definitely', 'even', 'dinner', 'people']
Burgers ['burger', 'food', 'place', 'burgers', 'one', 'fries', 'great', 'really', 'like', 'little', 'good', 'back', 'much', 'best', 'definitely', 'order', 'get', 'even', 'chicken', 'ordered']
